In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv("transformed_league_data.csv")
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,gameid,side,pick1,pick2,pick3,pick4,pick5,ban1,ban2,ban3,ban4,ban5,result,num_counters,num_counters_banned,PGA
0,0,0,0,FRA1TMNT1/450135,Blue,Rumble,Jarvan IV,Syndra,Sivir,Annie,Alistar,Azir,Irelia,NaN,NaN,0,0,1,1
1,1,1,1,FRA1TMNT1/450135,Red,Gnar,Rengar,Ahri,Graves,Janna,Rek'Sai,LeBlanc,Kassadin,NaN,NaN,1,0,1,1
2,2,2,2,FRA1TMNT1/450138,Blue,Lissandra,Rek'Sai,Xerath,Tristana,Janna,LeBlanc,Kassadin,Lee Sin,NaN,NaN,1,0,0,0
3,3,3,3,FRA1TMNT1/450138,Red,Rumble,Rengar,Syndra,Sivir,Annie,Gnar,Azir,Irelia,NaN,NaN,0,2,1,3
4,4,4,4,FRA1TMNT1/450168,Blue,Gnar,Lee Sin,Nidalee,Corki,Janna,Cassiopeia,Rek'Sai,Lissandra,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167135,167135,2917,2917,LOLTMNT06_96972,Red,Ambessa,Sejuani,Akali,Corki,Neeko,Skarner,Vi,Aurora,K'Sante,Jax,1,0,1,1
167136,167136,2918,2918,LOLTMNT06_96989,Blue,Gnar,Amumu,Jayce,Ezreal,Nautilus,Maokai,K'Sante,Viktor,Sion,Jax,1,0,0,0
167137,167137,2919,2919,LOLTMNT06_96989,Red,Malphite,Zyra,Corki,Kai'Sa,Leona,Skarner,Ambessa,Vi,Rell,Rakan,0,1,0,1
167138,167138,2920,2920,LOLTMNT06_96993,Blue,Ambessa,Zyra,Jayce,Ezreal,Nautilus,Varus,Sejuani,Kalista,Yone,Smolder,0,0,1,1


In [6]:
bin_edges = list(range(11))
bin_counts = df["PGA"].value_counts().reindex(bin_edges, fill_value=0).reset_index()
bin_counts.columns = ["PGA", "Count"]

fig = px.bar(
    bin_counts,
    x="PGA",
    y="Count",
    title="Distribution of Pre Game Advantages",
    labels={"PGA": "PGA Value", "Count": "Count"},
    text_auto=True
)

fig.update_layout(
    xaxis=dict(type="category"),
    bargap=0.05  
)

fig.show()
